<a href="https://colab.research.google.com/github/tae-h-yang/cs229/blob/main/CS_229_Transformers_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Remember to make a copy of this colab notebook before you start editing cells!

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.4 MB/s eta 0:00:00


In [2]:
!pip install tqdm

In [3]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-02-20 03:50:31--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2025-02-20 03:50:31 (47.1 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [4]:
# DO NOT MODIFY ANY OF THIS CODE

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

In [5]:
# DO NOT MODIFY ANY OF THIS CODE

# Hyperparameters
batch_sz = 16
context_length = 32
max_iterations = 30000
log_interval = 200
init_lr = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_steps = 200
embedding_dim = 64
num_heads = 4
num_blocks = 4
drop_prob = 0.0

# Load and prepare the data
torch.manual_seed(1337)
with open('input.txt', 'r', encoding='utf-8') as file:
    text_data = file.read()

unique_chars = sorted(set(text_data))
vocab_size = len(unique_chars)
char_to_index = {ch: i for i, ch in enumerate(unique_chars)}
index_to_char = {i: ch for i, ch in enumerate(unique_chars)}

def encode_text(s): return [char_to_index[c] for c in s]
def decode_text(l): return ''.join([index_to_char[i] for i in l])

# Split data for training and validation
data_tensor = torch.tensor(encode_text(text_data), dtype=torch.long)
train_size = int(0.9 * len(data_tensor))
train_data, val_data = data_tensor[:train_size], data_tensor[train_size:]

def generate_batch(split):
    data_src = train_data if split == 'train' else val_data
    indices = torch.randint(0, len(data_src) - context_length, (batch_sz,))
    inputs = torch.stack([data_src[i:i + context_length] for i in indices])
    targets = torch.stack([data_src[i + 1:i + context_length + 1] for i in indices])
    return inputs.to(device), targets.to(device)

@torch.no_grad()
def evaluate_loss():
    model.eval()
    losses = {'train': [], 'val': []}
    for split in ['train', 'val']:
        for _ in range(eval_steps):
            batch_x, batch_y = generate_batch(split)
            _, batch_loss = model(batch_x, batch_y)
            losses[split].append(batch_loss.item())
    model.train()
    return {split: torch.tensor(losses[split]).mean().item() for split in losses}

In [9]:
# YOU WILL CHANGE CODE IN THIS CELL
# Implement a Transformer model with PyTorch. Fill out the provided skeleton.

class SelfAttention(nn.Module):
    def __init__(self, head_dim):
        super().__init__()
        # TODO: initialize key, query, and value as linear layers. Set bias=False
        # self.key_proj = ...
        # self.query_proj = ...
        # self.value_proj = ...
        self.key_proj = nn.Linear(embedding_dim, head_dim, bias=False)
        self.query_proj = nn.Linear(embedding_dim, head_dim, bias=False)
        self.value_proj = nn.Linear(embedding_dim, head_dim, bias=False)

        self.register_buffer('mask', torch.tril(torch.ones(context_length, context_length)))
        self.dropout = nn.Dropout(drop_prob)

    def forward(self, x):
        # B, T, C = ...
        # keys, queries, values = ...
        B, T, C = x.shape
        keys = self.key_proj(x)
        queries = self.query_proj(x)
        values = self.value_proj(x)

        scores = (queries @ keys.transpose(-2, -1)) * (C ** -0.5)
        scores = scores.masked_fill(self.mask[:T, :T] == 0, float('-inf'))
        # TODO: apply softmax and dropout
        # attention_weights = ...
        # attention_weights = ...
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)

        return attention_weights @ values

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, num_heads, head_dim):
        super().__init__()
        self.heads = nn.ModuleList([SelfAttention(head_dim) for _ in range(num_heads)])
        self.output_proj = nn.Linear(embedding_dim, embedding_dim)
        self.dropout = nn.Dropout(drop_prob)

    def forward(self, x):
        # TODO: combine multiple attention heads
        # x = ...
        x = torch.cat([head(x) for head in self.heads], dim=-1)

        return self.dropout(self.output_proj(x))

class FeedForwardLayer(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(emb_dim, 4 * emb_dim),
            nn.ReLU(),
            nn.Linear(4 * emb_dim, emb_dim),
            nn.Dropout(drop_prob)
        )

    def forward(self, x):
        return self.layers(x)

class TransformerBlock(nn.Module):
    def __init__(self, emb_dim, num_heads):
        super().__init__()
        # TODO: initialize the multihead self attention, feed forward layer, and two layernorms
        # self.attention = ...
        # self.feed_forward = ...
        # self.norm1 = ...
        # self.norm2 = ...
        self.attention = MultiHeadSelfAttention(num_heads, emb_dim // num_heads)
        self.feed_forward = FeedForwardLayer(emb_dim)
        self.norm1 = nn.LayerNorm(emb_dim)
        self.norm2 = nn.LayerNorm(emb_dim)

    def forward(self, x):
        # TODO: implement the forward logic
        # x = ...
        # x = ...
        x = x + self.attention(self.norm1(x))
        x = x + self.feed_forward(self.norm2(x))

        return x

class TransformerLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # self.token_embeddings = ...
        # self.position_embeddings = ...
        self.token_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.position_embeddings = nn.Embedding(context_length, embedding_dim)

        self.transformer_blocks = nn.Sequential(*[TransformerBlock(embedding_dim, num_heads) for _ in range(num_blocks)])
        self.final_norm = nn.LayerNorm(embedding_dim)
        self.head = nn.Linear(embedding_dim, vocab_size)

    def forward(self, idx, targets=None):
        # B, T = ...
        # tok_emb = ...
        B, T = idx.shape
        tok_emb = self.token_embeddings(idx)

        pos_emb = self.position_embeddings(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.final_norm(self.transformer_blocks(x))
        logits = self.head(x)

        if targets is None:
            return logits, None

        logits = logits.view(B * T, vocab_size)
        # targets = ...
        # loss = ...
        targets = targets.view(B * T)
        loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate_text(self, idx, max_tokens):
        for _ in range(max_tokens):
            idx_cond = idx[:, -context_length:]
            logits, _ = self(idx_cond)
            probs = F.softmax(logits[:, -1, :], dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_token), dim=1)
        return idx

In [10]:
# DO NOT MODIFY ANY OF THIS CODE

# Initialize and train the model
model = TransformerLanguageModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=init_lr)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
best_val_loss = float('inf')
no_progress = 0
max_patience = 10

for step in tqdm(range(max_iterations)):
    if step % log_interval == 0 or step == max_iterations - 1:
        current_losses = evaluate_loss()
        print(f"Step {step}: train loss {current_losses['train']:.4f}, val loss {current_losses['val']:.4f}")
        scheduler.step(current_losses['val'])

        if current_losses['val'] < best_val_loss:
            best_val_loss = current_losses['val']
            no_progress = 0
        else:
            no_progress += 1

    batch_x, batch_y = generate_batch('train')
    _, batch_loss = model(batch_x, batch_y)
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()

start_context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_output = decode_text(model.generate_text(start_context, max_tokens=1000)[0].tolist())

with open("output.txt", "w", encoding="utf-8") as out_file:
    out_file.write(generated_output)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
  0%|          | 0/30000 [00:00<?, ?it/s]

Step 0: train loss 4.3551, val loss 4.3576


  1%|          | 206/30000 [00:11<1:31:32,  5.42it/s]

Step 200: train loss 2.5030, val loss 2.4985


  1%|▏         | 405/30000 [00:18<1:29:20,  5.52it/s]

...

Step 29600: train loss 1.4248, val loss 1.6084


 99%|█████████▉| 29809/30000 [18:24<00:30,  6.30it/s]

Step 29800: train loss 1.4231, val loss 1.6386


100%|██████████| 30000/30000 [18:31<00:00, 27.00it/s]

Step 29999: train loss 1.4201, val loss 1.6320


In [18]:
# DO NOT MODIFY ANY OF THIS CODE

# Generate from the model
with torch.no_grad():
    context = torch.tensor(encode_text("JULIET: "), dtype=torch.long).unsqueeze(0).to(device)
    generated_text = decode_text(model.generate_text(context, max_tokens=200)[0].tolist())
    print(generated_text)

JULIET: whole burness, on me dear litts of his pain:
If it not, God, horsek sleep not teeds will full man of your call lam.
Your contempt starve their give and mildly.
Is note, but this? Do nobly he wind,
It 
